- Load necessary module

In [167]:
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import multinomial
from collections import OrderedDict

- Load model that defined in channel realization

In [67]:
mat = io.loadmat('temp.mat')
model = {}
model['H'] = mat['H']
model['Wopt'] = mat['Wopt']
model['Fopt'] = mat['Fopt']
model['Ns'] = 1 ## need to change nb of stream in channel realization, this case 1
model['Nt'] = mat['Nt'][0,0]
model['Nr'] = mat['Nr'][0,0]
model['Nc'] = mat['Nc'][0,0]
model['Nray'] = mat['Nray'][0,0]
model['realization'] = mat['realization'][0,0]
model['At'] = mat['At']
model['Ar'] = mat['Ar']
model['Nrf'] = 2

Load values of phase shifter included:
- nb_bit: number of bit
- nb_ps: number of phase shifter in the system
- nb_state: number of state in each phase shifter (2^nb_bit)

In [3]:
nb_bit = 2
nb_ps = model['Nt']*model['Nrf']
nb_ps
nb_state = 2**nb_bit

In [4]:
for key in model:
    print(key)

At
Fopt
H
Ns
Ar
Nr
Wopt
Nray
Nrf
Nc
Nt
realization


In [40]:
def phase_shifter(N):
    """
    Define phase shifter with N bits resolution
    
    Arguments:
    N -- number of bits resolutions
    
    Returns:
    y -- python dictionnary of phase shifter: index of state and value of phase
    """
    nb_state = 2**N
    w = np.exp(1j*2*np.pi/nb_state)
    ps = np.zeros(nb_state,dtype=complex)
    for i in range(nb_state):
        ps[i] = w**i
    return np.around(ps, decimals=2)

In [41]:
phase_value = phase_shifter(nb_bit)
phase_value

array([ 1.+0.j,  0.+1.j, -1.+0.j, -0.-1.j])

### Initialize phase shifter 
dimension is: NtxNrfxN_state

In [42]:
# Initialize phase shifter system
ps_system_proba = np.ones((model['Nt'],model['Nrf'],nb_state))*(1/nb_state)
# for i in range(nb_ps):
#     ps_system[i] = -11/

In [43]:
print(ps_system_proba.shape)
ps_system_proba

(64, 2, 4)


array([[[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25,

In [44]:
elements = [1, 2, 3, 4]
print(np.random.choice(elements, 10, p=ps_system_proba[0,0,:])) #phase_value
np.random.choice(phase_value, 1, p=ps_system_proba[0,0,:])

[1 3 2 4 2 1 1 4 4 4]


array([-1.+0.j])

In [45]:
ps_system_proba[0,0,:]

array([0.25, 0.25, 0.25, 0.25])

### Step 1: Randomly generate S candidate analog beamformers

In [79]:
S = 200
F_rf = np.zeros((S,model['Nt'],model['Nrf']),dtype=complex)
for i in range(S):
    for j in range(model['Nt']):
        for k in range(model['Nrf']):
            F_rf[i,j,k] = np.random.choice(phase_value, 1, p=ps_system_proba[j,k,:])

### Step 2: Compute S corresponding digital precoders Fbb based on the effective channel Heq = HFrf

In [80]:
channel_eq = np.zeros((S,model['Nr'],model['Nrf']),dtype=complex)
F_bb = np.zeros((S,model['Nrf'],model['Ns']),dtype=complex)
channel_index = 0
for i in range(S):
    channel_eq[i,:,:] =  model['H'][:,:,channel_index]@F_rf[i,:,:]
    U, s, V = np.linalg.svd(channel_eq[i,:,:], full_matrices=True)
    V = V.conj().T
    F_bb[i,:,:] = V[0:model['Nt'],0:model['Ns']]

In [81]:
F_rf.shape

(200, 64, 2)

In [73]:
F_bb.shape

(200, 2, 1)

In [56]:
channel_eq.shape

(200, 16, 2)

### Step 3: Calculate the achievable sum-rate

In [84]:
## Assume an optimize combining at Receiver
Wopt = np.zeros((model['Nr'],model['Ns']),dtype=complex)
U, s, V = np.linalg.svd(model['H'][:,:,channel_index], full_matrices=True)
V = V.conj().T
# Fopt[:,:,reali] = V[0:Nt,0:Ns]
Wopt = U[0:model['Nr'],0:model['Ns']]
    

In [148]:
# SNR_dB = np.arange(-35,10,5)
# SNR = 10**(SNR_dB/10)
# smax = SNR.shape[0]
SNR = 10**(10/10)
R = np.zeros((S))

for i in range(S):
    R[i] = np.log2(np.linalg.det(np.eye(model['Ns']))+SNR/model['Ns']*np.linalg.pinv(Wopt)@model['H'][:,:,channel_index]@F_rf[i,:,:]@F_bb[i,:,:]@F_bb[i,:,:].conj().T@F_rf[i,:,:].conj().T@model['H'][:,:,channel_index].conj().T@Wopt).real


In [149]:
R

array([12.41687569, 12.17879324, 12.45857315, 12.71576242, 12.66714872,
       12.42252569, 12.29083104, 11.42861497, 11.41751666, 12.13078948,
       10.70304817, 13.11951118, 12.63239294,  8.1362591 , 13.15886758,
        9.97612054, 13.61553919, 13.11112659, 13.71064335, 12.6875507 ,
       10.08570746, 11.00050714, 13.54171531, 13.17446275, 11.40429897,
       13.36996242, 11.2895382 , 11.73028035, 10.44167064, 12.54640355,
       12.6365979 , 11.62695371, 12.94987384, 13.0872801 , 11.83585636,
       12.32905878, 11.3646456 , 10.54214789, 13.63559468, 11.33249931,
        8.61074   ,  9.83221926, 10.77478797, 13.68829704, 13.05365116,
       13.05881052, 11.67510199, 11.62351825, 12.30029987, 14.12037492,
       13.1801765 , 10.52092659, 11.44241367,  9.18635436, 11.67001136,
       11.90861421, 11.71703229, 11.99182521, 11.91478702, 12.48164562,
       11.4951693 , 11.64259101, 13.00646743, 11.04186195,  9.97922411,
       13.32207059, 11.40975289, 11.8759346 , 11.58409468, 12.13

### Step 4: Sort R in a descend order

In [202]:
nb_elite = 40

In [197]:
R[49]

14.12037491709517

In [203]:
R_sort_index = np.argsort(R)[::-1][:nb_elite]

In [204]:
R_sort_index

array([ 49, 131,  18,  43, 103, 118,  38,  16, 120,  22, 111, 124,  25,
       102,  65,  79, 156, 160,  50,  23,  14,  11, 163,  17,  33, 142,
        45,  44, 178,  78,  62, 104, 130, 164,  32, 146, 182,  88, 194,
       117])

In [211]:
R_max_index = R_sort_index[0]
R_max_index

49

In [205]:
print(R[R_sort_index])

[14.12037492 13.89559266 13.71064335 13.68829704 13.66089872 13.64081961
 13.63559468 13.61553919 13.59011902 13.54171531 13.51748673 13.47457156
 13.36996242 13.36920087 13.32207059 13.25489538 13.21832575 13.20165971
 13.1801765  13.17446275 13.15886758 13.11951118 13.11158574 13.11112659
 13.0872801  13.08679901 13.05881052 13.05365116 13.03564913 13.01479686
 13.00646743 13.00188184 12.98173214 12.97233232 12.94987384 12.93452344
 12.92784426 12.90078937 12.88388772 12.8352944 ]


In [213]:
F_elites = F_rf[R_sort_index,:,:]

(40, 64, 2)

### Step 5: Select elites as Frf_1, Frf_2, ..., Frf_elite

In [214]:
F_elites = F_rf[R_sort_index,:,:]